# Ch_06_학습 관련 기술들

6장에서는 신경망의 학습 효율을 높이기 위한 방법들을 설명하고자 한다.

설명할 내용들은 다음과 같다.

최적의 가중치 매개변수를 찾기 위한 최적화 방법.

초기 가중치 매개변수 설정에 관한 방법.

오버피팅 조정하는 방법.

배치 정규화.

6.1 매개변수 갱신

신경망 학습의 원리는 손실함수라는 지표를 정의한 다음에 손실함수 값을 가장 적게 하는 최적의 가중치 매개변수를 구하는 것이었다.

이처럼 어떤 목적함수의 최솟값을 찾는 문제를 수학에서는 최적화(Optimization)라고 한다.

하지만, 실제 feasible set의 영역이 매우 넓고, 변수 또한 많기 때문에 쉽게 계산 되지 않는다.

앞 단원에서 매개변수에 대한 손실함수의 기울기인 Gradient를 이용한 SGD(Stochastic Gradient Descent,확률적 경사 하강법)을 공부했다.

SGD는 처음 최적화를 풀 때 가장 자연스러운 방법이지만, 한계가 존재한다. 장단점을 살펴 본 후 보완할 수 있는 방법을 살펴보자.

6.1.1 모험가 이야기

어떤 모험가가 앞을 볼 수 없는 상태에서 현재의 위치로부터 가장 낮은 곳으로 가기 위해선 어떻게 해야할까?

지금 있는 곳에서 가장 땅의 기울기가 크게 기울어지는 곳으로 가면 될 것 같다.

6.1.2 SGD

SGD의 기본 원리를 복기 해보자.

현재의 가중치 행렬을 $\mathbf{W}$라 하면, 학습률 $\eta$에 대해 새로운 가중치를 다음과 같이 업데이트 한다.

[식 6.1]$~~~~\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{ \partial{L}}{\partial{W}} $

간단한 SGD를 구현해보자.

In [1]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads): #성분 별이 아닌 행렬 통째로 업데이트 함.
        for key in params.keys():
            params[key] -= self.lr * grads[key]

코드를 살펴보면, SGD라는 클래스 속의 update라는 Method는 params, grads라는 것을 인수로 받는데, params['W1']과 같이 행렬형태로 저장이 되기 때문에 업데이트가 행렬 통째로 됨을 짐작 할 수 있다.

이 SGD 클래스를 이용하여 앞에서의 신경망에서 매개변수 갱신 부분에 사용하면 된다.

6.1.3 SGD의 단점

개념 자체가 자연스럽고, 계산도 그리 복잡하지 않음에도 불구하고 SGD가 널리 쓰이지 않는 이유가 있다.

다음과 같은 식을 생각해보자.

[식 6.2]
![](data/images/e%206.2.png)
위의 식을 그래프로 그리면 다음과 같다.
![](data/images/fig%206-1.png)
이 때 [식 6.2]의 각 점에서의 Gradient를 살펴보면,
![](data/images/fig%206-2.png)
즉, y축 방향은 크게 감소하는데 x축 방향은 크게 바뀌지 않는다. 그리고 Global minimum 값은 (0,0)에서 나타나는게 자명하지만, Gradient는 모두 (0,0)을 가리키진 않는다. 이런 함수에 대해 SGD를 한번 적용해보자.

초기값을 (-7,2)로 했을 때 SGD 적용 결과는
![](data/images/fig%206-3.png)
즉, global minimum인 (0,0)으로 가긴 하지만, 지그재그 형태로 매우 많은 반복횟수를 거쳐 감을 알 수 있다.

이런 지그재그 현상이 일어나는 근본적인 원인은 각 점에서의 Gradient의 방향이 Global Minimum의 방향이 아니기 때문에 일어난다.

이런 현상을 개선하기 위해 모멘텀(Momentum), AdaGard, Adam 3가지 방법을 소개한다.

6.1.4 모멘텀

모멘텀은 물리에서 '운동량'과 관련이 있는 개념이다. 왜 모멘텀이란 이름이 붙었는지에 대한 철학은 다음에 다루어보도록 하고,

모멘텀 기법은 다음과 같이 업데이트를 한다.

$\mathbf{v} \leftarrow \alpha\mathbf{v} - \eta\frac{\partial{L}}{\partial{W}} \quad$ :속도에 해당하는 개념

$\mathbf{W}\leftarrow\mathbf{W} + \mathbf{v}$ 

일반적인 SGD에서는 $k$번째 스텝에서의 descent direction을 $k$번째 weight에 의한 L의 gradient direction으로 결정했다면,

모멘텀 기법은 $k$번째 스텝에서 descent direction을 $k-1$번째의 속도에 관련된 부분과 $k$번째의 gradient 방향을 모두 고려하겠다는 것이다.

그림을 그려보면 다음과 같다.

![](data/images1/momentum.png)
이 때 최초의 속도($\mathbf{v}_{0}$)는 gradient와 같은 값으로 설정한다.

모멘텀 기법을 구현해보자.

In [ ]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momnetum = momentum
        self.v = None
    
    def update(self, params, grads): 
        if self.v is None: #초기 속도를 설정하는데.... 뭐지
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
                
        for key in paras.key():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

모멘텀 기법을 사용하여 [식 6-2]의 최솟값을 구해보면,
![](data/images/fig%206-5.png)
단순한 SGD를 풀었을 때보다 조금 더 부드럽게 접근을 한다.

6.1.5 AdaGrad

SGD에서의 Learning Rate를 무엇으로 할것인지 또한 중요한 문제가 된다.

학습률을 정하는 방법은 여러가지가 있는데, 그 중 학습을 진행하면서 학습률을 점점 줄여가는 방법인 '학습률 감소'라는 방법이 있다.

이에 관련된 방법이 바로 AdaGrad이다.

AdaGrad는 각각의 매개변수에 적응하여(Adaptive) 학습률을 결정한다.

때문에 Adaptive Gradient 를 줄여 AdaGrad라고 이름을 붙인듯 하다.

AdaGrad의 갱신 방법을 수식으로 나타내면 다음과 같다.

[식 6.5]
$\mathbf{h} \leftarrow \mathbf{h} + \frac{\partial{L}}{\partial{W}} \odot \frac{\partial{L}}{\partial{W}}$

[식 6.6]
$\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{1}{\sqrt{\mathbf{h}}} \odot \frac{\partial{L}}{\partial{\mathbf{W}}}$

이 식을 설명하기 위해 간단한 최적화 문제를 생각해보자. 

$f:\mathbb{R}^{n} \rightarrow \mathbb{R}$인 함수의 최솟값을 구한다고 생각해보자.

이 경우의 SGD의 업데이트 방식을 살펴보면, 

$\mathbf{x}_{k+1} = \mathbf{x}_{k} - \eta \cdot \nabla f(\mathbf{x}_{k})$

이를 벡터로 표현하면,

$\begin{pmatrix} {x_{1}}^{(k+1)} \\ \vdots \\ {x_{n}}^{(k+1)} \end{pmatrix} = \begin{pmatrix} {x_{1}}^{(k)} \\ \vdots \\ {x_{n}}^{(k)} \end{pmatrix}- \eta \begin{pmatrix} (\nabla f(\mathbf{x}_{k}))_{1} \\ \vdots \\ (\nabla f(\mathbf{x}_{k}))_{n} \end{pmatrix}$

이 때, $(\nabla f(\mathbf{x}_{k}))_{i} = g_{k,i}$ 라고 쓰면, 업데이트의 성분별 표현은 

${x_{i}}^{(k+1)} = {x_{i}}^{(k)} - \eta \cdot g_{k,i}$로 나타낼 수 있다.

그렇다면 AdaGrad의 성분별 업데이트의 식은 다음과 같다.

${x_{i}}^{(k+1)} = {x_{i}}^{(k)} - \frac{\eta}{\sqrt{(\sum_{t=1}^k {g^{2}_{t,i}})+\epsilon}} \cdot g_{k,i}$

(이 때 $\epsilon$은 분모가 0이 되는 경우를 방지하기 위한 임의의 양수라 생각. 보통 $10^{-8}$을 택한다. 여기서는 우선 0이라고 생각하자.)

(즉 k번째 까지의 ${\nabla f(x_{t})}^{2}$을 다 더한 값이 분모로 들어가므로 매 순간 learning rate 값은 바뀌고, 그 분모의 값이 점점 증가 하므로 learning rate의 값은 점점 감소하게 된다.)

이를 다시 벡터로 나타내면,

$\begin{pmatrix} {x_{1}}^{(k+1)} \\ \vdots \\ {x_{n}}^{(k+1)} \end{pmatrix} = \begin{pmatrix} {x_{1}}^{(k)} \\ \vdots \\ {x_{n}}^{(k)} \end{pmatrix}- \begin{pmatrix} \frac{\eta}{\sqrt{\sum_{t=1}^k {g^{2}}_{t,1}}} \cdot g_{k,1} \\ \vdots \\ \frac{\eta}{\sqrt{\sum_{t=1}^k {g^{2}_{t,n}}}} \cdot g_{k,n} \end{pmatrix}$

이를 다시 벡터간의 원소 곱을 나타내는 $\odot$ 이 기호를 이용하면,

$\begin{pmatrix} {x_{1}}^{(k+1)} \\ \vdots \\ {x_{n}}^{(k+1)} \end{pmatrix} = \begin{pmatrix} {x_{1}}^{(k)} \\ \vdots \\ {x_{n}}^{(k)} \end{pmatrix} - \begin{pmatrix} \frac{\eta}{\sqrt{\sum_{t=1}^k {g^{2}_{t,1}}}} \\ \vdots \\ \frac{\eta}{\sqrt{\sum_{t=1}^k {g^{2}_{t,n}}}} \end{pmatrix} \odot \begin{pmatrix} g_{k,1} \\ \vdots \\ g_{k,n} \end{pmatrix}$

이제 $\begin{pmatrix} \sum_{t=1}^k {g^{2}_{t,1}} \\ \vdots \\ \sum_{t=1}^k {g^{2}_{t,n}} \end{pmatrix} = \mathbf{G}_{k}$라고 하고 다시 쓰면,

$\mathbf{x}_{k+1} = \mathbf{x}_{k} - \frac{\eta}{\sqrt{\mathbf{G}_{k}}} \odot \nabla f(\mathbf{x}_{k}) $

여기서 $\mathbf{G}_{k} = \mathbf{h}$ 라고 보면, 식 6.5와 식 6.6이 설명이 된다.

AdaGrad의 장점은 매 단계마다 스스로 학습률을 조정해준다.

하지만 단점으로는 학습률의 분모 부분에는 계속해서 양수가 더해지므로 숫자가 커진다. 따라서 학습률은 단계가 지나면 지날수록 작아지기 때문에, 계속해서 학습을 하는 경우에는 학습률이 매우 작아져 더 이상 학습이 안될 수도 있다.

이 단점의 보완으로 RMSProp이라는 방법이 있는데, 이 방법은 나중에 SGD를 정리할 때 다시 공부해보자.

이제 AdaGrad를구현해보자.

In [4]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h =None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key])+1e-7)
            

이 AdaGrad 방법을 이용하여 [식 6.2]의 최적화 문제를 풀어보면 경로는 다음과 같다.

![](data/images/fig%206-6.png)
처음에 y축 방향으로의 감소률이 크므로 단계가 지날 때마다 y축 방향으로의 감소율이 조정이 되 지그재그같은 움직임이 사라진다.

6.1.6 Adam

모멘텀은 각 점에서의 Gradient에다 그 이전의 움직임까지 고려해서 다음 방향을 결정하는 방법이고,

AdaGrad는 학습이 진행 됨에 따라 학습률이 조절되는 방법이었다.

Adam은 간단히 말하면 모멘텀과 AdaGrad의 방법(정확히는 RMSprop)을 융합한 개념이다.

간단히 RMSprop에 대해 살펴보자.

AdaGrad의 핵심 개념은 학습률의 분모 부분에 그 이전까지의 Gradient의 제곱합을 더한 값의 루트 값이 있었다.

RMSprop의 기본 개념은 단계가 지남에 따라 초기의 gradient값의 영향력은 줄이고, 전 단계의 gradient의 영향력을 더 반영하겠다는 것이다.

즉, $\sum_{t=1}^k {g^{2}_{t,i}}$ 대신에, $\ 0.9 \cdot (\sum_{t=1}^{k-1} {g^{2}_{t,i}}) + 0.1 \cdot {g^{2}}_{k,i}$로 사용하겠다는 뜻이다.

어쨋든 ${g^{2}}_{1,i}\ ,{g^{2}}_{2,i}\ ,...,\ {g^{2}}_{k-1,i}$의 평균으로 나타나지므로 다시 적으면,

$E[{g^{2}}_{i}]_{k} = 0.9 \cdot E[{g^{2}}_{i}]_{k-1} + 0.1 \cdot {g^{2}}_{k,i}$

이 개념을 이용하여 RMSprop의 성분별 업데이트 형식을 살펴보면,

${x_{i}}^{(k+1)} = {x_{i}}^{(k)} - \frac{\eta}{\sqrt{E[{g^{2}}_{i}]_{k}+\epsilon}} \cdot g_{k,i}$

이제 Adam에 대해서 생각해보자. 

먼저 AdaGrad와, RMSprop의 알고리즘 업데이트를 다시 살펴보면,

AdaGrad : ${x_{i}}^{(k+1)} = {x_{i}}^{(k)} - \frac{\eta}{\sqrt{(\sum_{t=1}^k {g^{2}_{t,i}})+\epsilon}} \cdot g_{k,i}$

RMSprop : ${x_{i}}^{(k+1)} = {x_{i}}^{(k)} - \frac{\eta}{\sqrt{E[{g^{2}}_{i}]_{k}+\epsilon}} \cdot g_{k,i}$

Adam은 이미 언급했듯이 모멘텀과 AdaGrad를 융합한 개념이다.

이전의 Gradient의 지수평균을 계속 저장한 것을 모멘텀으로 사용하면서, RMSprop에서의 기울기의 제곱 부분을 지수평균으로 저장하여 학습률도 함께 업데이트하는 방식이다.

momentum에 해당하는 부분을 $m_{k}=({m^{(k)}}_{1}, ... ,{m^{(k)}_{n}})$, 학습률에 해당하는 부분을 $ v_{k}=({v^{(k)}}_{1}, ... ,{v^{(k)}_{n}})$라 하자.

이 때 각 $m_{k}$와 $v_{k}$는 다음과 같이 업데이트 된다.

$m_{k}= \beta_{1} m_{k-1} + (1 - \beta_{1})\nabla f(x_{k}) \quad $: momentum에 해당하는 부분 

$v_{k}= \beta_{2} v_{k-1} + (1 - \beta_{2})(\nabla f(x_{k}))^{2} \quad $ : 학습률에 해당하는 부분

(이 때 $\beta_{1}, \beta_{2} $는 사람이 정해주는 Hyperparameter(초모수)에 해당하는 수, 경험적으로  $\beta_{1}=0.9, \beta_{2}=0.999$ )

그런데 0-step에서 $m_{0} = v_{0} = \mathbf{0}$ 이렇게 초기화가 될텐데, 그렇게 되면 처음 몇 step은 $\mathbf{0}$에 굉장히 편향되어있게 된다.

이 편향된 부분을 조정하기 위해 간단한 과정을 거쳐 다음의 값을 도출해낸다.

$ \hat{m}_{k} = \frac{m_{k}}{1 - {\beta_1}^k} $

$ \hat{v}_{k} = \frac{v_{k}}{1 - {\beta_2}^k} $

이 값을 이용하여 업데이트를 진행했을 때 성분별 업데이트 형식은 다음과 같다.

${x_{i}}^{(k+1)} = {x_{i}}^{(k)} - \frac{\eta}{\sqrt{{\hat{v}^{(k)}}_{i}}+\epsilon} \cdot {\hat{m}}^{(k)}_{i}$

벡터로 표시하면,

$ \mathbf{x_{k+1}} = \mathbf{x_{k}} - \frac{\eta}{\sqrt{\mathbf{\hat{v}_{k}}}+\epsilon} \odot \mathbf{\hat{m}_{k}}$

